In [2]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [5]:
# Create vine_table dataframe
vine_table = df.select('review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine','verified_purchase')
vine_table.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3EFW2STIYIY0I|          1|            2|          2|   N|                Y|
|R12NR0R5A9F7FT|          5|            0|          0|   N|                Y|
|R1LSH74R9XAP59|          2|            0|          1|   N|                Y|
|R1QXUNTF76K7L6|          2|            0|          0|   N|                Y|
|R2F7DR75PS8NKT|          5|            0|          0|   N|                Y|
|R2C1DJSCC8UFS6|          3|            0|          0|   N|                Y|
|R1AXGS1W4YFXMX|          1|            0|          2|   N|                Y|
|R1XU1B93402SYJ|          1|            1|          1|   N|                Y|
|R2U432NB3OPVR0|          5|            0|          0|   N|                Y|
|R3R6FIMIOQ5SP9|          5|            0|          0|   N|     

In [9]:
# filter for total votes greater than 20
df_20votes = vine_table.filter(vine_table.total_votes >= 20)
df_20votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26CC1STPSUHBS|          1|           27|         39|   N|                N|
| R7K83D59T9KWS|          1|           16|         26|   N|                N|
|R2ATG5BPVILHUM|          1|           16|         25|   N|                N|
|R1Q71TAM5ZOYTK|          3|           57|         61|   N|                N|
| RR8HYI7C2XKMU|          5|           25|         25|   N|                Y|
|R1VDCSJE4CJJIR|          1|           88|         97|   N|                Y|
|R2PZMSDKOO1Q6W|          1|           29|         43|   N|                Y|
|R39OOYN0DOLINK|          3|           91|        111|   N|                N|
|R1PLNHJZ8BKIBG|          1|           44|         46|   N|                Y|
|R14G4YQXL17REH|          1|            4|         28|   N|     

In [11]:
# filter helpful_votes/total_votes greater than 50%
helpful_df = df_20votes.filter((df_20votes['helpful_votes']) / (df_20votes['total_votes']) >= 0.5)
helpful_df.show(5)
helpful_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26CC1STPSUHBS|          1|           27|         39|   N|                N|
| R7K83D59T9KWS|          1|           16|         26|   N|                N|
|R2ATG5BPVILHUM|          1|           16|         25|   N|                N|
|R1Q71TAM5ZOYTK|          3|           57|         61|   N|                N|
| RR8HYI7C2XKMU|          5|           25|         25|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



17762

In [12]:
# Filtering out the paid vine reviews
vine_review_df = helpful_df.filter(helpful_df['vine'] == 'Y')

vine_review_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28EIR3Z6RRUNN|          2|           50|         60|   Y|                N|
|R2YO2RESXZEWOF|          4|           24|         25|   Y|                N|
| R8GEE2EW3D6S9|          3|           49|         54|   Y|                N|
|R33W9V1MX6WKPS|          4|           47|         53|   Y|                N|
|R24E3W2ZRIFG8C|          5|           91|         93|   Y|                N|
|R2OR3KIFFG4YWT|          4|           75|         81|   Y|                Y|
|R1RLJAAFOA5MMS|          2|           25|         27|   Y|                N|
| RVYHD7IZ7W99B|          1|           78|         78|   Y|                N|
|R12EV95D2T56HA|          5|           37|         37|   Y|                N|
|R296NVZGV4VZLP|          5|           41|         44|   Y|     

In [13]:
# Filter out the unpaid vine reviews
unpaid_vine_review_df = helpful_df.filter(helpful_df['vine'] == 'N')

unpaid_vine_review_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26CC1STPSUHBS|          1|           27|         39|   N|                N|
| R7K83D59T9KWS|          1|           16|         26|   N|                N|
|R2ATG5BPVILHUM|          1|           16|         25|   N|                N|
|R1Q71TAM5ZOYTK|          3|           57|         61|   N|                N|
| RR8HYI7C2XKMU|          5|           25|         25|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [22]:
# Number of paid vine reviews
vine_paid_count = vine_review_df.count()
print(f" There are {vine_paid_count} vine reviews")

# Total vine program reviews
five_star_df = vine_review_df.filter(vine_review_df['star_rating'] == '5')
five_star_vine_reviewcount = five_star_df.count()
print(f" There are a total of {five_star_vine_reviewcount} vine program reviews")

#Percentage of reviews that are 5 stars
percentage = round(((five_star_vine_reviewcount / vine_paid_count) * 100), 0)
print(f" {percentage} of the total reviews are 5 star reviews")

 There are 248 vine reviews
 There are a total of 102 vine program reviews
 41.0 of the total reviews are 5 star reviews


In [24]:
# Number of unpaid vine reviews
vine_unpaid_count = unpaid_vine_review_df.count()
print(f" There are {vine_unpaid_count} non-vine reviews")

# Total vine program reviews
unpaid_five_star_df = unpaid_vine_review_df.filter(unpaid_vine_review_df['star_rating'] == '5')
unpaid_five_star_vine_reviewcount = unpaid_five_star_df.count()
print(f" There are a total of {unpaid_five_star_vine_reviewcount} non-vine program reviews")

#Percentage of reviews that are 5 stars
percentage_unpaid = round(((unpaid_five_star_vine_reviewcount / vine_unpaid_count) * 100), 0)
print(f" {percentage_unpaid} of the total non-vine reviews are 5 star reviews")

 There are 17514 non-vine reviews
 There are a total of 5154 non-vine program reviews
 29.0 of the total non-vine reviews are 5 star reviews
